In [66]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data
import importlib
import livenet
importlib.reload(livenet)
core = livenet.core
import simple_log
simple_log.level = simple_log.LogLevel.INFO
from simple_log import LOG
import math
from matplotlib import pyplot as plt
%matplotlib
plt.ion()
var = 42
print(torch.__version__)
np.set_printoptions(precision=3)
%precision 4



Using matplotlib backend: TkAgg
2.2.1+cu121


'%.4f'

In [67]:
downscale = (14, 14)
train_x, train_y = livenet.datasets.to_plain(*livenet.datasets.get_mnist_train(), downscale=downscale, to_odd=True, to_gray=True)
n_hidden = 2

# downscale = (8, 8)
# train_x, train_y = livenet.datasets.to_plain(*livenet.datasets.get_cifar10_train(), downscale=downscale, to_odd=False, to_gray=True)
# n_hidden = 12

n_out = np.max(train_y.numpy()) + 1



In [85]:
context = core.livenet.Context()
network = livenet.nets.create_perceptron(train_x.shape[1], n_hidden, n_out, context)
batch_iterator = livenet.gen_utils.batch_iterator(train_x, train_y, batch_size=1000)
criterion = livenet.nets.criterion_classification_n
optimizer = livenet.nets.create_optimizer(network)
trainer = livenet.net_trainer.NetTrainer(network, batch_iterator, criterion, optimizer, epoch_size=1)

LiveNet


In [91]:
network.context.regularization_l1 = 0.01
optimizer.learning_rate = 0.001
trainer.step(5000)


Iˈ0.000 0.905 = 0.741+0.164reg params=8 livenet/net_trainer.py:93
Iˈ0.004 0.872 = 0.708+0.164reg params=8 livenet/net_trainer.py:93
Iˈ0.007 0.918 = 0.753+0.164reg params=8 livenet/net_trainer.py:93
Iˈ0.009 0.919 = 0.755+0.164reg params=8 livenet/net_trainer.py:93
Iˈ0.012 0.945 = 0.781+0.164reg params=8 livenet/net_trainer.py:93
Iˈ0.014 0.938 = 0.774+0.164reg params=8 livenet/net_trainer.py:93
Iˈ0.017 0.955 = 0.791+0.164reg params=8 livenet/net_trainer.py:93
Iˈ0.020 0.895 = 0.731+0.164reg params=8 livenet/net_trainer.py:93
Iˈ0.022 0.917 = 0.753+0.164reg params=8 livenet/net_trainer.py:93
Iˈ0.024 0.898 = 0.734+0.164reg params=8 livenet/net_trainer.py:93
Iˈ0.027 0.920 = 0.756+0.164reg params=8 livenet/net_trainer.py:93
Iˈ0.029 0.975 = 0.811+0.164reg params=8 livenet/net_trainer.py:93
Iˈ0.031 0.874 = 0.710+0.164reg params=8 livenet/net_trainer.py:93
Iˈ0.033 1.002 = 0.837+0.164reg params=8 livenet/net_trainer.py:93
Iˈ0.035 0.953 = 0.789+0.164reg params=8 livenet/net_trainer.py:93
Iˈ0.036 0.

In [92]:
pred = network(train_x)
pred_bin = np.argmax(pred.detach().numpy(), axis=1, keepdims=True)
diff = train_y - pred_bin
f"accuracy {len(diff[diff == 0]) / len(diff): .3f}"


'accuracy  0.767'

In [93]:
for name, p in sorted(network.named_parameters(),key=lambda x: len(x[0]) + ord(x[0][0]) / 1000. + ord(x[0][1]) / 1e6):
    print(name, f"{p.item():.1f}")



D0 -6.3
D1 6.3
N1 6.5
N1->D0 0.2
N1->D1 -2.9
S1->N1 -2.0
S2->N1 3.7
S3->N1 1.5


In [94]:
%matplotlib
plt.ion()

def param_amount(param):
    val = len(param["params"])
    return val

def param_max(param):
    val = np.max(np.abs(param["params"]))
    return val

def param_picker1(param):
    try:
        val0 = param["params"]["N1->D0"].item()
    except KeyError:
        val0 = 0.
    return val0

def get_param_values(history, picker):
    values = []
    for entry in history:
        values.append(picker(entry))
    return values

plt.figure(figsize=(16, 9))
values = get_param_values(trainer.history, param_picker1)
plt.grid()
plt.plot(values)

#accum = core.stat_utils.AccumStat()
#accum.add_value(trainer.network.parameters())
#accum.plot()


Using matplotlib backend: TkAgg


D0 -14.3
D1 14.3
N1 11.9
N1->D0 -0.1
N1->D1 -3.0
S1->N1 -1.2
S2->N1 2.9
S3->N1 1.3


In [52]:
network._parameters


OrderedDict([('D0',
              Parameter containing:
              tensor(24.0669, requires_grad=True)),
             ('D1',
              Parameter containing:
              tensor(-24.0669, requires_grad=True)),
             ('N1',
              Parameter containing:
              tensor(13.0908, requires_grad=True)),
             ('S6->N1',
              Parameter containing:
              tensor(0.7347, requires_grad=True)),
             ('S8->N1',
              Parameter containing:
              tensor(-0.6229, requires_grad=True)),
             ('S9->N1',
              Parameter containing:
              tensor(-1.5580, requires_grad=True)),
             ('S11->N1',
              Parameter containing:
              tensor(-0.5902, requires_grad=True)),
             ('N1->D0',
              Parameter containing:
              tensor(-1.7404, requires_grad=True)),
             ('N1->D1',
              Parameter containing:
              tensor(1.5365, requires_grad=True))])

In [ ]:
core.livenet.export_onnx(network, "/home/spometun/model.onnx")
